In [2]:
import os
import pandas as pd

#### The next four blocks are for reading Tappy Keystroke data into dataframes. You can skip these and load processed dataframe files directly.

In [23]:
rows = []
colNames = ['BirthYear','Gender','Parkinsons','Tremors','DiagnosisYear','Sided','UPDRS','Impact','Levadopa','DA','MAOB','Other']
users = []

userFolder = 'data/Users'

for userFilename in os.listdir(userFolder):
    users.append(userFilename[5:-4])

    f = open(os.path.join(userFolder, userFilename))
    lines = f.readlines()

    row = []
    col = []

    for line in lines:
        line = line[:-1]
        lineSplit = line.split(': ')
        
        row.append(lineSplit[1])
        col.append(lineSplit[0])
            
    if col == colNames:
        rows.append(row)
    else:
        print(userFilename+'read wrong.')

    f.close()

userDF = pd.DataFrame(rows, columns=colNames, index=users)

In [5]:
userDF.head()

,BirthYear,Gender,Parkinsons,Tremors,DiagnosisYear,Sided,UPDRS,Impact,Levadopa,DA,MAOB,Other
0EA27ICBLF,1952,Female,True,True,2000,Left,Don't know,Severe,True,True,False,False
0QAZFRHQHW,1959,Female,False,False,------,None,Don't know,------,False,False,False,False
0WTDIGPSBZ,1946,Female,False,False,------,None,Don't know,------,False,False,False,False
1HOEBIGASW,1944,Male,False,False,------,None,Don't know,------,False,False,False,False
1WMVCCU4RH,1953,Male,True,True,2017,Left,Don't know,Medium,False,False,False,False


In [21]:
rows = []
colNames = ['UserID','Date','Timestamp','Hand','HoldTime','Direction','LatencyTime','FlightTime']

dataFolder = 'data/TappyData'

for dataFilename in os.listdir(dataFolder):
    f = open(os.path.join(dataFolder, dataFilename))
    lines = f.readlines()

    row = []

    for line in lines:
        line = line[:-1]
        lineSplit = line.split('\t')
        
        rows.append(lineSplit[:-1])

    f.close()

dataDF = pd.DataFrame(rows, columns=colNames)

In [4]:
dataDF.head()

,UserID,Date,Timestamp,Hand,HoldTime,Direction,LatencyTime,FlightTime
0,0EA27ICBLF,160722,18:41:04.336,L,0101.6,LL,0234.4,0156.3
1,0EA27ICBLF,160722,18:42:14.070,L,0085.9,LL,0437.5,0359.4
2,0EA27ICBLF,160722,18:42:14.273,L,0078.1,LL,0210.9,0125.0
3,0EA27ICBLF,160722,18:42:14.617,L,0062.5,LL,0359.4,0281.3
4,0EA27ICBLF,160722,18:42:15.586,S,0125.0,LS,0187.5,0093.8


In [26]:
userDF.to_pickle('data/users.pkl')
dataDF.to_pickle('data/data.pkl')

#### Load processed dataframe

In [3]:
userDF = pd.read_pickle('data/users.pkl')
dataDF = pd.read_pickle('data/data.pkl')